In [10]:
import os
from datetime import datetime
import shutil
from glob import glob
import rioxarray as rxr
from rioxarray.exceptions import NoDataInBounds
import rasterio.features
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon
from tqdm import tqdm
import matplotlib.pyplot as plt
from IPython import display
from rasterio.errors import NotGeoreferencedWarning
from scipy.ndimage import gaussian_filter
import importlib.util
from rioxarray.merge import merge_arrays
from scipy.optimize import minimize
from scipy.signal import find_peaks
import warnings
import torch
import pandas as pd
import pickle
import cv2
from multiprocessing import Pool
warnings.filterwarnings("ignore", category=NotGeoreferencedWarning)
np.seterr(divide='ignore', invalid='ignore')
#import clear output
from IPython.display import clear_output
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
# istarmap.py for Python <3.8



def recreate_dir(path):
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)
    return path

def load_config(path):
    spec = importlib.util.spec_from_file_location("CFG", path)
    CFG = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(CFG)
    return CFG

def inline_chart(values, width=10):
    min_val = np.min(values)
    max_val = np.max(values)
    #if len(values) > width resample to 10 values
    if len(values) > width:
        values = np.array_split(values, width)
        values = [np.mean(v) for v in values]
    else:
        #pad with min value
        values = np.pad(values, (0, width-len(values)), "constant", constant_values=min_val)
    #normalize
    values = (values - min_val)/(max_val - min_val)
    chart = ""
    for value in values:
        if value >= 0. and value < 0.125:
            chart+="▁"
        elif value >= 0.125 and value < 0.25:
            chart+="▂"
        elif value >= 0.25 and value < 0.375:
            chart+="▃"
        elif value >= 0.375 and value < 0.5:
            chart+="▄"
        elif value >= 0.5 and value < 0.625:
            chart+="▅"
        elif value >= 0.625 and value < 0.75:
            chart+="▆"
        elif value >= 0.75 and value < 0.875:
            chart+="▇"
        elif value >= 0.875 and value <= 1.:
            chart+="▉"
    return chart

In [11]:
DATA_DIR = "data/grodzisko_20230111"
CFG = load_config(f"{DATA_DIR}/config.py").CALIBRATION

In [12]:
#configure logging to file
import logging
log_path = f"{DATA_DIR}/logs/calibration_{datetime.now().strftime('%d%m%Y%H%M%S')}.log"
os.makedirs(os.path.dirname(log_path), exist_ok=True)
logging.basicConfig(filename=log_path,level=logging.INFO, format='%(asctime)s %(levelname)-8s %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger(__name__)
logger.handlers.clear()
#logger.addHandler(logging.StreamHandler())
logger.info("Starting procedure")

In [13]:
TMP_DIR = f"{DATA_DIR}/tmp"
recreate_dir(TMP_DIR)
TIF_DEVIGNETTE_DIR = f"{DATA_DIR}/tif_devignette"
assert os.path.exists(TIF_DEVIGNETTE_DIR), "TIF_DEVIGNETT_DIR does not exist. Please run 2_devignetting.ipynb first."
GEOTIF_DEVIGNETTE_DIR = f"{DATA_DIR}/geotif_devignette"
assert os.path.exists(GEOTIF_DEVIGNETTE_DIR), "GEOTIF_DEVIGNETT_DIR does not exist. Please run 2_devignetting.ipynb first."

TIF_CAL_DIR = f"{DATA_DIR}/tif_cal"
GEOTIF_CAL_DIR = f"{DATA_DIR}/geotif_cal"

# PLOT_CLIP_DIR = f"{DATA_DIR}/plot_clip"
# PLOT_CAL_DIR = f"{DATA_DIR}/plot_cal"
CACHE_DIR = f"{DATA_DIR}/cache"
os.makedirs(CACHE_DIR, exist_ok=True)
# TEMP_OPTIM_DATASET_DIR = f"{DATA_DIR}/temp_optim_dataset"
# I_CLIP_DIR = f"{TEMP_OPTIM_DATASET_DIR}/i_clip"
# J_CLIP_DIR = f"{TEMP_OPTIM_DATASET_DIR}/j_clip"
# CLIP_MASK_DIR = f"{TEMP_OPTIM_DATASET_DIR}/clip_mask"

AssertionError: GEOTIF_DEVIGNETT_DIR does not exist. Please run 2_devignetting.ipynb first.

In [ ]:
if CFG.CACHE and os.path.exists(f"{CACHE_DIR}/footprints.pkl"):
    logger.info("Loading footprints from cache")
    with open(f"{CACHE_DIR}/footprints.pkl", "rb") as f:
        footprints = pickle.load(f)
else:
    logger.info("Reading footprints from geotifs")
    geometries = []
    names = []
    timestamps = []
    for path in tqdm(glob(f"{GEOTIF_DEVIGNETTE_DIR}/*.tif")):
        raster = rxr.open_rasterio(path)
        footprints = rasterio.features.shapes((raster != raster.rio.nodata).values.astype(np.int16), transform=raster.rio.transform())
        footprints = [Polygon(geom["coordinates"][0]).simplify(10) for geom, colval in footprints if colval == 1]
        assert len(footprints) == 1, "More than one footprint found"
        #example name DJI_20230303092123_0066_T.tif. Get timestamp
        timestamp = os.path.basename(path).split("_")[1]
        timestamp = datetime.strptime(timestamp, "%Y%m%d%H%M%S")
        timestamp = timestamp.timestamp()
        
        names.append(os.path.basename(path))
        geometries.append(footprints[0])
        timestamps.append(timestamp)
    timestamps = np.array(timestamps)
    timestamps = (timestamps - timestamps.min())
    footprints = gpd.GeoDataFrame({"name": names, "time": timestamps, "geometry": geometries})
    #write CRS
    footprints.crs = CFG.CRS
    with open(f"{CACHE_DIR}/footprints.pkl", "wb") as f:
        pickle.dump(footprints, f)

NameError: name 'CACHE_DIR' is not defined

In [ ]:
# def nan_gaussian_filter(arr, sigma):
#     """Apply gaussian filter to array while ignoring nans"""
#     V=arr.copy()
#     V[np.isnan(arr)]=0
#     VV=gaussian_filter(V,sigma=sigma)
#     W=0*arr.copy()+1
#     W[np.isnan(arr)]=0
#     WW=gaussian_filter(W,sigma=sigma)
#     Z=VV/WW
#     Z[np.isnan(arr)]=np.nan
#     return Z


In [ ]:

if CFG.CACHE and os.path.exists(f"{CACHE_DIR}/calibration_pairs.pkl"):
    logger.info("Loading cached pairs")
    with open(f"{CACHE_DIR}/calibration_pairs.pkl", "rb") as f:
        pairs_i, pairs_j, pairs_area, i_clips, j_clips, masks, diff_vars = pickle.load(f)
else:
    logger.info("Generating temprature global optimization dataset")
    #erode footprints
    if CFG.EROSION > 0:
        footprints["geometry"] = footprints["geometry"].buffer(-CFG.EROSION)
    # recreate_dir(TEMP_OPTIM_DATASET_DIR)
    # recreate_dir(I_CLIP_DIR)
    # recreate_dir(J_CLIP_DIR)
    # recreate_dir(CLIP_MASK_DIR)
    # recreate_dir(PLOT_CLIP_DIR)
    pairs_i = []
    pairs_j = []
    pairs_area = []
    i_clips = []
    j_clips = []
    masks = []
    diff_vars = []
    idx = 0
    for i in tqdm(range(len(footprints))):
        i_raster = rxr.open_rasterio(f"{GEOTIF_DEVIGNETTE_DIR}/{footprints.iloc[i]['name']}", masked=True)
        for j in range(i+1, len(footprints)):
            if footprints.iloc[i].geometry.intersects(footprints.iloc[j].geometry):
                intersection = footprints.iloc[i].geometry.intersection(footprints.iloc[j].geometry)
                if intersection.area < CFG.MIN_INTERSECTION_AREA:
                    logger.info(f"i ({i}), j ({j}): intersection area too small")
                    continue
                j_raster = rxr.open_rasterio(f"{GEOTIF_DEVIGNETTE_DIR}/{footprints.iloc[j]['name']}", masked=True)
                try:
                    i_clip = i_raster.rio.clip([intersection])
                    j_clip = j_raster.rio.clip([intersection])
                except NoDataInBounds:
                    logger.info(f"i ({i}), j ({j}): NoDataInBounds")
                    continue
                j_clip = j_clip.rio.reproject_match(i_clip)
                i_clip = i_clip.values[0]
                j_clip = j_clip.values[0]
                diff = i_clip - j_clip
                #variance of difference
                diff_var = np.nanvar(diff)
                #offset = np.nanmean(i_clip) - np.nanmean(j_clip)
                #mse = np.nanmean((i_clip - j_clip)**2)
                #rmse with offset compensated as a measure of pair alignment
                # offset = np.nanmean(i_clip) - np.nanmean(j_clip)
                
                #i_clip = nan_gaussian_filter(i_clip, sigma=CFG.GAUSS_SIGMA)
                #j_clip = nan_gaussian_filter(j_clip, sigma=CFG.GAUSS_SIGMA)
                # i_var = np.nanvar(i_clip)
                # j_var = np.nanvar(j_clip)
                # var = np.nanmean([i_var, j_var])
                # if np.isnan(var):
                #     logger.info(f"i ({i}), j ({j}): var is nan")
                #     continue
                mask = (~np.isnan(i_clip) & ~np.isnan(j_clip)).astype(np.int16)
                i_clip[np.isnan(i_clip)] = np.nanmean(i_clip)
                j_clip[np.isnan(j_clip)] = np.nanmean(j_clip)
                
                # mask_copy = mask.copy()
                # i_clip_copy = i_clip.copy()
                # j_clip_copy = j_clip.copy()
                
                i_clip = cv2.resize(i_clip, (CFG.CLIP_SIZE, CFG.CLIP_SIZE), interpolation=cv2.INTER_LINEAR)
                j_clip = cv2.resize(j_clip, (CFG.CLIP_SIZE, CFG.CLIP_SIZE), interpolation=cv2.INTER_LINEAR)
                mask = cv2.resize(mask, (CFG.CLIP_SIZE, CFG.CLIP_SIZE), interpolation=cv2.INTER_NEAREST)
                
                #assert i_clip, j_clip, mask dont have nans
                if np.isnan(i_clip).any():
                    logger.info(f"i ({i}), j ({j}): i_clip has nan")
                    continue
                if np.isnan(j_clip).any():
                    logger.info(f"i ({i}), j ({j}): j_clip has nan")
                    continue
                if np.isnan(mask).any():
                    logger.info(f"i ({i}), j ({j}): mask has nan")
                    continue
                ###############################
                # np.save(f"{I_CLIP_DIR}/{idx}.npy", i_clip_copy)
                # np.save(f"{J_CLIP_DIR}/{idx}.npy", j_clip_copy)
                # np.save(f"{CLIP_MASK_DIR}/{idx}.npy", mask_copy)
                ###############################
                # make plot with two images
                # fig, ax = plt.subplots(2, 2, figsize=(10, 10), dpi=50)
                # i_raster.plot(ax=ax[0][0])
                # ax[0][0].plot(*intersection.exterior.xy, color="red")
                # j_raster.plot(ax=ax[0][1])
                # ax[0][1].plot(*intersection.exterior.xy, color="red")
                # ax[1][0].imshow(i_clip, cmap="gray")
                # ax[1][1].imshow(j_clip, cmap="gray")
                # #save fig to file
                # fig.savefig(f"{PLOT_CLIP_DIR}/{i}_{j}.png")
                # plt.close()
                ##############################
                pairs_i.append(i)
                pairs_j.append(j)
                pairs_area.append(intersection.area)
                i_clips.append(i_clip)
                j_clips.append(j_clip)
                masks.append(mask)
                diff_vars.append(diff_var)
                idx+=1
                
                # idx += 1
    pairs_i = np.array(pairs_i)
    pairs_j = np.array(pairs_j)
    pairs_area = np.array(pairs_area)
    i_clips = np.array(i_clips)
    j_clips = np.array(j_clips)
    masks = np.array(masks)
    diff_vars = np.array(diff_vars)
    #pickle dump
    with open(f"{CACHE_DIR}/calibration_pairs.pkl", "wb") as f:
        pickle.dump((pairs_i, pairs_j, pairs_area, i_clips, j_clips, masks, diff_vars), f)

NameError: name 'CACHE_DIR' is not defined

In [ ]:
area_weights = pairs_area#np.emath.logn(10, pairs_area)
#normalize from 0 to 1
area_weights = area_weights/area_weights.max()#(area_weights - areights.min()) / (area_weights.max() - area_weights.min())
#var_weights = np.emath.logn(10, 1/diff_vars)
#var_weights = (var_weights - var_weights.min()) / (var_weights.max() - var_weights.min())
#weights = area_weights * var_weights
weights = area_weights

In [ ]:
#recreate_dir(PLOT_CLIP_DIR)

n_images = len(footprints)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pairs_i = torch.tensor(pairs_i, dtype=torch.int64, device=device) # (n_pairs,)
pairs_j = torch.tensor(pairs_j, dtype=torch.int64, device=device) # (n_pairs,)
pairs_area = torch.tensor(pairs_area, dtype=torch.float32, device=device) # (n_pairs,)
#pairs_var = torch.tensor(pairs_var, dtype=torch.float32, device=device) # (n_pairs,)
i_clips = torch.tensor(i_clips, dtype=torch.float32, device=device) # (n_pairs, width, height)
j_clips = torch.tensor(j_clips, dtype=torch.float32, device=device) # (n_pairs, width, height)
masks = torch.tensor(masks, dtype=torch.float32, device=device) # (n_pairs, width, height)
weights = torch.tensor(weights, dtype=torch.float32, device=device) # (n_pairs,)
pairs_n_pixels = torch.sum(masks, dim=(1, 2)) # (n_pairs,)
n_pixels = torch.sum(pairs_n_pixels) # (1,)
#n_pixels_weighted = torch.sum(pairs_n_pixels * weights) # (1,)
n_pairs = len(pairs_i) # (1,)

i_clips_mean = torch.sum(i_clips * masks, dim=(1, 2)) / pairs_n_pixels # (n_pairs,)
j_clips_mean = torch.sum(j_clips * masks, dim=(1, 2)) / pairs_n_pixels # (n_pairs,)
i_clips_var = torch.sum(masks * (i_clips - i_clips_mean[:, None, None]) ** 2, dim=(1, 2)) / pairs_n_pixels # (n_pairs,)
j_clips_var = torch.sum(masks * (j_clips - j_clips_mean[:, None, None]) ** 2, dim=(1, 2)) / pairs_n_pixels # (n_pairs,)
a_coefs = torch.ones(n_images, dtype=torch.float32, device=device, requires_grad=False) # (n_images,)
b_coefs = torch.zeros(n_images, dtype=torch.float32, device=device, requires_grad=True) # (n_images,)

optimizer = torch.optim.Adam([b_coefs], lr=CFG.INIT_LR)#([a_coefs, b_coefs], lr=CFG.INIT_LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=CFG.LR_PLATEAU_FACTOR, patience=CFG.LR_PLATEAU_PATIENCE, cooldown=CFG.LR_PLATEAU_COOLDOWN, threshold=CFG.LR_PLATEAU_THRESHOLD, verbose=True)
pairs_i_copy = pairs_i.clone()
pairs_j_copy = pairs_j.clone()
best_loss = np.inf
es_counter = 0
pixel_losses = []
mean_losses = []
var_losses = []
for epoch in (pbar := tqdm(range(CFG.EPOCHS))):
    #assert that pairs_i and pairs_j are equal to the original values
    assert torch.all(pairs_i == pairs_i_copy)
    assert torch.all(pairs_j == pairs_j_copy)
    pbar.set_description(f"Epoch {epoch}")
    optimizer.zero_grad()
    i_clips_cal = a_coefs[pairs_i, None, None] * i_clips + b_coefs[pairs_i, None, None] # (n_pairs, width, height)
    j_clips_cal = a_coefs[pairs_j, None, None] * j_clips + b_coefs[pairs_j, None, None] # (n_pairs, width, height)
    i_clips_cal_mean = torch.sum(i_clips_cal * masks, dim=(1, 2)) / pairs_n_pixels # (n_pairs,)
    j_clips_cal_mean = torch.sum(j_clips_cal * masks, dim=(1, 2)) / pairs_n_pixels # (n_pairs,)
    i_clips_cal_var = torch.sum(masks * (i_clips_cal - i_clips_cal_mean[:, None, None]) ** 2, dim=(1, 2)) / pairs_n_pixels # (n_pairs,)
    j_clips_cal_var = torch.sum(masks * (j_clips_cal - j_clips_cal_mean[:, None, None]) ** 2, dim=(1, 2)) / pairs_n_pixels # (n_pairs,)
    #assert that i_clips_cal_mean and j_clips_cal_mean do not contain any NaNs

    # #group i_clips_cal_mean by pairs_i and calculate the mean
    # if epoch % 50 == 0:
    #     xs = []
    #     ys = []
    #     for k in range(n_images):
    #         for mean_val in i_clips_cal_mean.detach().cpu().numpy()[pairs_i == k]:
    #             xs.append(k)
    #             ys.append(mean_val)
    #         for mean_val in j_clips_cal_mean.detach().cpu().numpy()[pairs_j == k]:
    #             xs.append(k)
    #             ys.append(mean_val)
    #     plt.plot(xs, ys, ".")
    #     plt.show()
    
    pixel_loss = torch.sum((i_clips_cal - j_clips_cal)**2 * masks * weights[:, None, None])/n_pixels
    #pixel_loss = torch.mean((i_clips_cal_mean - j_clips_cal_mean)**2)
    # mean_loss = 0.000001  *(0.5/n_pairs)*(
    #                 torch.sum((i_clips_cal_mean - i_clips_mean)**2 * weights[:, None, None]) + 
    #                 torch.sum((j_clips_cal_mean - j_clips_mean)**2 * weights[:, None, None])
    #             )
    #var_loss =  0.1   *(0.5/n_pairs)*(torch.sum(torch.abs(i_clips_cal_var - i_clips_var)**1)   + torch.sum(torch.abs(j_clips_cal_var - j_clips_var)**1))

    loss = pixel_loss# + mean_loss# + var_loss
    loss.backward()
    optimizer.step()
    scheduler.step(loss)
    
    if best_loss-loss.item() > CFG.ES_MIN_DELTA:
        best_loss = loss.item()
        best_a_coefs = a_coefs.detach().cpu().numpy()
        best_b_coefs = b_coefs.detach().cpu().numpy()
        es_counter = 0
    else:
        es_counter += 1
        if es_counter > CFG.ES_PATIENCE:
            print("Early stopping")
            # for i_clip, j_clip, i, j, mask, a_i, b_i, a_j, b_j, i_clip_cal_mean, j_clip_cal_mean in zip(tqdm(i_clips_cal.detach().cpu().numpy()), j_clips_cal.detach().cpu().numpy(), pairs_i.detach().cpu().numpy(), pairs_j.detach().cpu().numpy(), masks.detach().cpu().numpy(), a_coefs[pairs_i].detach().cpu().numpy(), b_coefs[pairs_i].detach().cpu().numpy(), a_coefs[pairs_j].detach().cpu().numpy(), b_coefs[pairs_j].detach().cpu().numpy(), i_clips_cal_mean.detach().cpu().numpy(), j_clips_cal_mean.detach().cpu().numpy()):
            #     fig, ax = plt.subplots(1,2, dpi=50)
            #     ax[0].imshow(i_clip * mask)
            #     ax[0].set_title(f"{i} {i_clip_cal_mean}")
            #     ax[1].imshow(j_clip * mask)
            #     ax[1].set_title(f"{j} {j_clip_cal_mean}")
            #     fig.savefig(f"{PLOT_CLIP_DIR}/{i}_{j}.png")
            #     plt.close(fig)
            break

    pixel_losses.append(pixel_loss.item())
    #mean_losses.append(mean_loss.item())
    #var_losses.append(var_loss.item())

    pbar.set_postfix({
        "pixel_loss": pixel_loss.item(), 
        #"mean_loss": mean_loss.item(),
        #"var_loss": var_loss.item(),
        "loss": f"{loss.item()}", 
        "a_mean": a_coefs.mean().item(), 
        "b_mean": b_coefs.mean().item()})

NameError: name 'footprints' is not defined

In [ ]:
#plot losses with log scale
plt.figure(figsize=(15, 5))
plt.plot(pixel_losses, label="pixel_loss")
#plt.plot(mean_losses, label="mean_loss")
plt.plot(var_losses, label="var_loss")
#plt.yscale("log")
plt.legend()
plt.show()

NameError: name 'pixel_losses' is not defined

<Figure size 1500x500 with 0 Axes>

In [ ]:

recreate_dir(GEOTIF_CAL_DIR)
recreate_dir(TIF_CAL_DIR)
def save_calib(name, a, b):
    geotif = rxr.open_rasterio(f"{GEOTIF_DEVIGNETTE_DIR}/{name}", masked=True)
    geotif.data = geotif.data * a + b
    geotif.rio.to_raster(f"{GEOTIF_CAL_DIR}/{name}")
    

with Pool(6) as pool:
    #result = pool.istarmap(save_calib, zip(footprints["name"].values, best_a_coefs, best_b_coefs))
    result = list(tqdm(pool.istarmap(save_calib, zip(footprints["name"].values, best_a_coefs, best_b_coefs)), total=30))

In [ ]:
recreate_dir(GEOTIF_CAL_DIR)
recreate_dir(TIF_CAL_DIR)
for name, a, b in zip(tqdm(footprints["name"]), best_a_coefs, best_b_coefs):
    geotif = rxr.open_rasterio(f"{GEOTIF_DEVIGNETTE_DIR}/{name}", masked=True)
    geotif.data = geotif.data * a + b
    geotif.rio.to_raster(f"{GEOTIF_CAL_DIR}/{name}")
    # tif = rxr.open_rasterio(f"{TIF_DEVIGNETTE_DIR}/{name}", masked=True)
    # tif.data = tif.data * a + b
    # tif.rio.to_raster(f"{TIF_CAL_DIR}/{name}")

100%|██████████| 262/262 [01:16<00:00,  3.42it/s]


In [24]:
from time import sleep

import multiprocessing.pool as mpp
def istarmap(self, func, iterable, chunksize=1):
    """starmap-version of imap
    """
    self._check_running()
    if chunksize < 1:
        raise ValueError(
            "Chunksize must be 1+, not {0:n}".format(
                chunksize))

    task_batches = mpp.Pool._get_tasks(func, iterable, chunksize)
    result = mpp.IMapIterator(self)
    self._taskqueue.put(
        (
            self._guarded_task_generation(result._job,
                                          mpp.starmapstar,
                                          task_batches),
            result._set_length
        ))
    return (item for chunk in result for item in chunk)
mpp.Pool.istarmap = istarmap

from multiprocessing import Pool
import tqdm    

def foo(a, b):
    for _ in range(int(50e6)):
        pass
    return a, b    

with Pool(4) as pool:
    iterable = [(i, 'x') for i in range(10)]
    for _ in tqdm.tqdm(pool.istarmap(foo, iterable),
                        total=len(iterable)):
        pass
    
def multiprocess(func, iterable, n_jobs=6):
    with Pool(n_jobs) as pool:
        for _ in tqdm.tqdm(pool.istarmap(foo, iterable),
                        total=len(iterable)):
            pass

100%|██████████| 10/10 [00:07<00:00,  1.42it/s]


In [ ]:

from multiprocessing import Pool
import tqdm    

def fun(a, b):
    sleep(4)
    return f"{a} {b}"
l1 = [1,2,3, 4, 5, 6, 7, 8, 9, 10]
l2 = ["foo", "bar", "baz", "foo", "bar", "baz", "foo", "bar", "baz", "foo"]
with Pool(6) as pool:
    #result = pool.istarmap(save_calib, zip(footprints["name"].values, best_a_coefs, best_b_coefs))
    result = list(tqdm(pool.istarmap(fun, zip(l1, l2)), total=10))